In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

URL = open("url.txt").read().strip()

# =========================
# CHROME OPTIONS (WINDOWS)
# =========================
options = Options()
options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 20)
driver.get(URL)
time.sleep(4)

# =========================
# SAFE GET
# =========================
def safe_get(by, selector, attr=None):
    try:
        el = driver.find_element(by, selector)
        return el.get_attribute(attr) if attr else el.text
    except:
        return None

# =========================
# BASIC PLACE DATA
# =========================
data = {}
data["name"] = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf"))
).text

data["rating"] = safe_get(By.CSS_SELECTOR, "span.F7nice span")
data["review_count"] = safe_get(By.CSS_SELECTOR, "span.F7nice span:nth-child(2)")
data["address"] = safe_get(By.CSS_SELECTOR, "button[data-item-id='address'] div.Io6YTe")
data["phone"] = safe_get(By.CSS_SELECTOR, "button[data-item-id*='phone'] div.Io6YTe")
data["website"] = safe_get(By.CSS_SELECTOR, "a[data-item-id='authority'] div.Io6YTe")

print("\n===== PLACE INFO =====")
for k, v in data.items():
    print(f"{k}: {v}")

# =========================
# OPEN REVIEWS
# =========================
review_btn = wait.until(EC.element_to_be_clickable((
    By.XPATH,
    "//button[contains(@aria-label,'Đánh giá') or contains(@aria-label,'Reviews')]"
)))
review_btn.click()

review_btn.click()
time.sleep(3)

# =========================
# SCROLL REVIEWS
# =========================
scroll_box = wait.until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf")
))

last_height = 0
for _ in range(15):  # tăng nếu muốn nhiều review hơn
    driver.execute_script(
        "arguments[0].scrollTop = arguments[0].scrollHeight",
        scroll_box
    )
    time.sleep(2)

# =========================
# READ REVIEWS
# =========================
reviews = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")
review_list = []

for r in reviews:
    try:
        user = r.find_element(By.CSS_SELECTOR, "div.d4r55").text
    except:
        user = None

    try:
        rating = r.find_element(By.CSS_SELECTOR, "span.kvMYJc").get_attribute("aria-label")
    except:
        rating = None

    try:
        text = r.find_element(By.CSS_SELECTOR, "span.wiI7pd").text
    except:
        text = None

    try:
        time_review = r.find_element(By.CSS_SELECTOR, "span.rsqaWe").text
    except:
        time_review = None

    review_list.append({
        "user": user,
        "rating": rating,
        "time": time_review,
        "text": text
    })

# =========================
# OUTPUT
# =========================
print(f"\n===== REVIEWS ({len(review_list)}) =====")
for r in review_list[:5]:
    print(r)

driver.quit()



===== PLACE INFO =====
name: Hot Beans Coffee & Cuisine
rating: None
review_count: None
address: 2A2 Chu Mạnh Trinh, Phường Sài Gòn, Quận 1, Thành phố Hồ Chí Minh 71006, Vietnam
phone: 
website: 

===== REVIEWS (160) =====
{'user': 'jimvanny', 'rating': '5 stars', 'time': '2 months ago', 'text': 'A beautiful spot not too far of a walk from the Saigon Liberty Central Riverside hotel.  The food was great.  I had the croque monsieur.  It was delicious.  My wife had phở gà kho.  The noodles were so good!  We shared a side of roasted …'}
{'user': 'Amanda Hovenga', 'rating': '5 stars', 'time': '2 months ago', 'text': 'We had a whole floor to ourselves at 9.30 on Wednesday morning. The coffee was strong and life giving. I tried my first salted coffee and it was delicious as seen in photo. The Viennese breakfast was light and scrumptious so I had some desert! So affordable!'}
{'user': 'Marc Finkelstein', 'rating': '1 star', 'time': 'a month ago', 'text': 'I went here on the strength of its re